# Demo of End-to-End Imaging Loop

This notebook demonstrates the AstroViper imaging loop, which implements a tclean-like workflow with:
- Major/minor cycle iteration control
- Visibility-domain residual calculation
- Hogbom deconvolution
- Convergence visualization

## Setup

First, we import the required modules and generate synthetic visibility data with point sources.

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pylab as pl

# AstroViper imports
from astroviper.core.imaging.imaging_utils.standard_gridding_example import generate_ms4_with_point_sources
from astroviper.core.imaging.imaging_utils.iteration_control import (
    IterationController,
    ReturnDict,
    plot_convergence_history,
    ConvergencePlots,
)
from astroviper.core.imaging.imager import run_imaging_loop

## Generate Synthetic MSv4 with Point Sources

We use `generate_ms4_with_point_sources` to create a measurement set with 4 point sources.
This function:
1. Downloads Antennae field UV coverage (if not cached)
2. Creates simulated point sources at random positions
3. Returns the MS4 DataTree along with image parameters

In [ ]:
# Generate 4 point sources with 1 Jy each
nsources = 4
source_fluxes = np.ones(nsources)  # 1 Jy each

sources, npix, cell, ms4 = generate_ms4_with_point_sources(nsources, source_fluxes)
# Reset weights and flags, since we have put in a simulated source
ms4["WEIGHT"] = xr.ones_like(ms4["WEIGHT"])
ms4["FLAG"] = xr.zeros_like(ms4["FLAG"])

print(f"Generated MS4 with {nsources} point sources")
print(f"Image size: {npix} x {npix} pixels")
print(f"Cell size: {cell}")
print(f"\nMS4 dimensions: {dict(ms4.dims)}")
print(f"Polarizations: {list(ms4.coords['polarization'].values)}")

## Polarization Handling

The Antennae dataset has 2 polarizations (XX, YY). The imaging loop natively supports 2-pol data:
- 2-pol linear (XX, YY) → Stokes I, Q
- 2-pol circular (RR, LL) → Stokes I, V

For unpolarized point sources, Stokes Q will be zero (XX ≈ YY).

In [ ]:
# For 2-pol linear (XX, YY), it produces Stokes I and Q

print(f"MS4 polarizations: {list(ms4.coords['polarization'].values)}")
print(f"VISIBILITY shape: {ms4['VISIBILITY'].shape}")
print(f"  -> Will produce Stokes ['I', 'Q'] (2-pol linear)")

In [ ]:
# Show source positions
print("Simulated source positions and fluxes:")
for k in range(len(sources[0])):
    print(f"  Source {k+1}: pixel ({sources[0][k]}, {sources[1][k]}), flux = {sources[2][k]:.2f} Jy")

## Configure the Imaging Loop

Now we set up the imaging parameters.

Key parameters:
- `cell_size`: Pixel size in radians (from the MS4 generation)
- `image_size`: Image dimensions in pixels
- `nmajor`: Maximum number of major cycles (set to 3 for this demo)
- `niter`: Maximum total CLEAN iterations
- `threshold`: Stopping threshold in Jy

In [ ]:
# Convert cell size to radians
cell_rad = cell.to('rad').value

# Configure imaging parameters
params = {
    # Image geometry - use the npix from generated data
    "image_size": (npix, npix),
    "cell_size": (-cell_rad, cell_rad),  # RA typically negative
    
    # Gridding
    "support": 7,
    "oversampling": 100,
    
    # Deconvolution
    "algorithm": "hogbom",
    "gain": 0.1,
    "niter": 10000,           # Max total iterations
    "threshold": 0.01,      # Stop at 10 mJy
    
    # Major cycle control - CAPPED AT 3 FOR THIS DEMO
    "nmajor": 3,
    "cyclefactor": 1.5,
    "minpsffraction": 0.05,
    "maxpsffraction": 0.8,
    
    # Spectral/polarization mode
    "chan_mode": "cube",
    "corr_type": "linear",  # XX, YY -> Stokes I, Q
}

print("Imaging parameters configured:")
print(f"  Image: {params['image_size'][0]} x {params['image_size'][1]} pixels")
print(f"  Cell size: {abs(params['cell_size'][0]) * 206265:.2f} arcsec")
print(f"  Max major cycles: {params['nmajor']}")
print(f"  Max iterations: {params['niter']}")
print(f"  Threshold: {params['threshold']*1000:.1f} mJy")

## Run the Imaging Loop

Now we execute the imaging loop. This will:
1. Create the PSF (Stokes I only)
2. For each major cycle:
   - Grid visibilities to make dirty image
   - Convert to Stokes basis (I, Q for 2-pol linear)
   - Run Hogbom deconvolution (minor cycles)
   - Degrid model to predict model visibilities
   - Compute residual visibilities
3. Track convergence throughout

In [ ]:
# Run the imaging loop
model, residual, return_dict, controller = run_imaging_loop(
    ms4=ms4,
    params=params,
    initial_model=None,
    output_dir=".",
)

## Examine Results

Let's look at the final model and residual images.

In [ ]:
print(f"Model shape: {model.shape}  # (chan, stokes, y, x)")
print(f"Residual shape: {residual.shape}")
print()
print(f"Controller state:")
print(f"  Major cycles completed: {controller.major_done}")
print(f"  Total iterations: {controller.total_iter_done}")
print(f"  Stop code: {controller.stopcode}")
print(f"  Stop reason: {controller.stopdescription}")

In [ ]:
# Plot the Stokes I model and residual
fig, axes = pl.subplots(1, 3, figsize=(15, 5))

# Extract Stokes I (index 0) from shape (chan, stokes, y, x)
model_I = model[0, 0, :, :].real
residual_I = residual[0, 0, :, :].real

# Model (Stokes I)
im0 = axes[0].imshow(model_I, origin='lower')
axes[0].set_title(f'Model (Stokes I)\nTotal flux: {np.sum(model_I):.3f} Jy')
pl.colorbar(im0, ax=axes[0], label='Jy/pixel')

# Residual (Stokes I)
im1 = axes[1].imshow(residual_I, origin='lower')
axes[1].set_title(f'Residual (Stokes I)\nPeak: {np.max(np.abs(residual_I)):.4f} Jy')
pl.colorbar(im1, ax=axes[1], label='Jy/pixel')

# Restored (Model + Residual, approximate)
restored = model_I + residual_I
im2 = axes[2].imshow(restored, origin='lower')
axes[2].set_title('Restored (Model + Residual)')
pl.colorbar(im2, ax=axes[2], label='Jy/pixel')

pl.tight_layout()
pl.show()

In [ ]:
# Compare recovered sources with input
print("Source comparison:")
print("="*60)
print(f"{'Input Position':<20} {'Input Flux':<15} {'Model Value':<15}")
print("-"*60)

for k in range(len(sources[0])):
    y, x = int(sources[0][k]), int(sources[1][k])
    input_flux = sources[2][k]
    model_value = model_I[y, x]
    print(f"({y:3d}, {x:3d})          {input_flux:10.3f} Jy    {model_value:10.3f} Jy")

## Convergence Visualization

The `IterationController` tracks convergence history in the `ReturnDict`. We can visualize this using `plot_convergence_history` or `ConvergencePlots`.

The convergence plot shows:
- **Peak Residual** (blue, left y-axis): How the peak residual decreases over iterations
- **Model Flux** (red, right y-axis): How the total model flux increases as sources are cleaned

In [ ]:
# Print ReturnDict contents to see what was tracked
print("ReturnDict contents:")
print(f"  Number of entries: {len(return_dict.data)}")
print(f"  Keys: {list(return_dict.data.keys())}")

# Show first entry details
if return_dict.data:
    first_key = list(return_dict.data.keys())[0]
    print(f"\nFirst entry (key={first_key}):")
    for field, value in return_dict.data[first_key].items():
        if isinstance(value, list) and len(value) > 3:
            print(f"  {field}: [{value[0]}, {value[1]}, ..., {value[-1]}] ({len(value)} values)")
        else:
            print(f"  {field}: {value}")

In [ ]:
# Interactive convergence plot using HoloViews
# This creates a dual y-axis plot with selectors for Stokes parameter and channel
convergence_plot = plot_convergence_history(
    return_dict,
    time=0,
    stokes='I',
    chan=0,
    width=800,
    height=400,
)

convergence_plot

In [ ]:
# Alternative: Use ConvergencePlots class for more control
plotter = ConvergencePlots(return_dict)
plotter.plot_history(time=0, stokes='I', chan=0)

## Summary

This notebook demonstrated:

1. **Synthetic data generation** using `generate_ms4_with_point_sources`
2. **Native 2-pol support** - no need to expand XX,YY to 4-pol; produces Stokes I,Q directly
3. **Imaging loop execution** with `run_imaging_loop` from `astroviper.core.imaging.imager`
4. **Results inspection** - model, residual, and restored images
5. **Convergence visualization** using `plot_convergence_history`

The imaging loop implements a complete tclean-like workflow with:
- Visibility-domain residuals (for accuracy)
- Major/minor cycle architecture
- Adaptive cycle thresholding
- Comprehensive convergence tracking
- Native support for 2-pol and 4-pol data

In [ ]:
# Final summary
print("="*60)
print("IMAGING LOOP SUMMARY")
print("="*60)
print(f"Input: {nsources} point sources, {np.sum(source_fluxes):.1f} Jy total")
print(f"Output Stokes: I, Q (from 2-pol linear XX, YY)")
print(f"Output model flux (Stokes I): {np.sum(model[:, 0, :, :]):.3f} Jy")
print(f"Final peak residual: {np.max(np.abs(residual[:, 0, :, :])):.4f} Jy")
print(f"Major cycles: {controller.major_done}")
print(f"Total iterations: {controller.total_iter_done}")
print(f"Stop reason: {controller.stopdescription}")
print("="*60)

## Compare grid2image_spheroid_msv4 vs Manual Gridding

In [ ]:
# Imports for comparison
from astroviper.core.imaging.imaging_utils.standard_grid import (
    standard_grid_numpy_wrap_input_checked,
    grid2image_spheroid_ms4,
    create_prolate_spheroidal_kernel_1D,
)
from astroviper.core.imaging.imaging_utils.gcf_prolate_spheroidal import create_prolate_spheroidal_kernel
from astroviper.core.imaging.ifft import ifft_uv_to_lm

In [ ]:
# Parameters for both methods
support = 7
oversampling = 100
image_size = np.array([npix, npix])
cell_size_arr = np.array([-cell_rad, cell_rad])
n_pol = ms4.dims['polarization']
ms4["WEIGHT"] = xr.ones_like(ms4["WEIGHT"])
ms4["FLAG"] = xr.zeros_like(ms4["FLAG"])

dirty_combined = np.zeros([1, n_pol, npix, npix], dtype=float)
grid2image_spheroid_ms4(
    vis=ms4,
    resid_array=dirty_combined,
    pixelincr=cell_size_arr,
    support=support,
    sampling=oversampling,
    dopsf=False,
    column='VISIBILITY',
    chan_mode='continuum'
)

print("grid2image_spheroid_ms4:")
print(f"  Output shape: {dirty_combined.shape}")
print(f"  Peak: {np.max(np.abs(dirty_combined[:,0])):.6f}")

In [ ]:

cgk_1D = create_prolate_spheroidal_kernel_1D(oversampling, support)

# Extract data from ms4
vis_data = ms4.VISIBILITY.data
uvw = ms4.UVW.data
freq_chan = ms4.coords["frequency"].values

# Create weight array (all ones, matching vis shape except last dim=1)
dims = ms4.dims
weight = np.ones([dims["time"], dims["baseline_id"], dims["frequency"], 1])

# Grid the visibilities
grid = np.zeros([1, n_pol, npix, npix], dtype=np.complex128)
sumwt = np.zeros([1, n_pol], dtype=np.float64)

#weight = ms4.WEIGHT.data.copy()   # (70, 86, 1, 2) — matches vis shape
#flag = ms4.FLAG.data
#weight[flag] = 0.0
#nan_uvw = np.isnan(uvw[:, :, 0]) | np.isnan(uvw[:, :, 1])
#weight[nan_uvw] = 0.0

standard_grid_numpy_wrap_input_checked(
    vis_data=vis_data,
    grid=grid,
    sum_weight=sumwt,
    uvw=uvw,
    weight=weight,
    freq_chan=freq_chan,
    cgk_1D=cgk_1D,
    image_size=image_size,
    cell_size=cell_size_arr,
    oversampling=oversampling,
    support=support,
    complex_grid=True,
    do_psf=False,
    chan_mode='continuum'
)

print(f"  Grid shape: {grid.shape}")
print(f"  Sum weights: {sumwt}")

In [ ]:
kernel, corrTerm = create_prolate_spheroidal_kernel(oversampling, support, image_size)

dirty_manual = (
    ifft_uv_to_lm(grid, axes=[2, 3])
    / corrTerm
    * image_size[0]
    * image_size[1]
    / sumwt[:, :, np.newaxis, np.newaxis]  # broadcast sumwt to image shape
)

# Take real part (should be real for Stokes I)
dirty_manual = dirty_manual.real

print("Method 2 (manual steps):")
print(f"  Output shape: {dirty_manual.shape}")
print(f"  Peak: {np.max(np.abs(dirty_manual[:,0])):.6f}")

In [ ]:
# ============================================================
# Compare the two methods
# ============================================================
diff = dirty_combined[:,0] - dirty_manual[:,0]
max_diff = np.max(np.abs(diff))
rel_diff = max_diff / np.max(np.abs(dirty_combined))

print("="*60)
print("COMPARISON: grid2image_spheroid_ms4 vs Manual Steps")
print("="*60)
print(f"Max absolute difference: {max_diff:.2e}")
print(f"Max relative difference: {rel_diff:.2e}")
print(f"Arrays match: {np.allclose(dirty_combined, dirty_manual, rtol=1e-10, atol=1e-14)}")
print("="*60)

# Visual comparison
fig, axes = pl.subplots(1, 3, figsize=(15, 4))

# Combined method (pol 0)
im0 = axes[0].imshow(dirty_combined[0, 0, :, :], origin='lower')
axes[0].set_title('grid2image_spheroid_ms4\n(pol 0)')
pl.colorbar(im0, ax=axes[0])

# Manual method (pol 0)
im1 = axes[1].imshow(dirty_manual[0, 0, :, :], origin='lower')
axes[1].set_title('Manual gridding\n(pol 0)')
pl.colorbar(im1, ax=axes[1])

# Difference
im2 = axes[2].imshow(diff[0,:, :], origin='lower', cmap='RdBu_r')
axes[2].set_title(f'Difference\nmax={max_diff:.2e}')
pl.colorbar(im2, ax=axes[2])

pl.tight_layout()
pl.show()